In [ ]:
import time

# Calcul Tsne, détermination des clusters et calcul ARI entre vrais catégorie et n° de clusters
def ARI_fct(features, ) :
    time1 = time.time()
    num_labels=len(l_cat)
    tsne = manifold.TSNE(n_components=2, perplexity=30, n_iter=2000, 
                                 init='random', learning_rate=200, random_state=42)
    X_tsne = tsne.fit_transform(features)
    # Détermination des clusters à partir des données après Tsne 
    cls = cluster.KMeans(n_clusters=num_labels, n_init=100, random_state=42)
    cls.fit(X_tsne)
    ARI = np.round(metrics.adjusted_rand_score(y_cat_num, cls.labels_),4)
    time2 = np.round(time.time() - time1,0)
    print("ARI : ", ARI, "time : ", time2)
    return ARI, X_tsne, cls.labels_


# visualisation du Tsne selon les vraies catégories et selon les clusters
def TSNE_visu_fct(X_tsne, y_cat_num, labels, ARI) :
    fig = plt.figure(figsize=(15,6))
    
    ax = fig.add_subplot(121)
    scatter = ax.scatter(X_tsne[:,0],X_tsne[:,1], c=y_cat_num, cmap='Set1')
    ax.legend(handles=scatter.legend_elements()[0], labels=l_cat, loc="best", title="Categorie")
    plt.title('Représentation des tweets par catégories réelles')
    
    ax = fig.add_subplot(122)
    scatter = ax.scatter(X_tsne[:,0],X_tsne[:,1], c=labels, cmap='Set1')
    ax.legend(handles=scatter.legend_elements()[0], labels=set(labels), loc="best", title="Clusters")
    plt.title('Représentation des tweets par clusters')
    
    plt.show()
    print("ARI : ", ARI)


def cnn_redimensionner_image(image):
    '''
    Image à redimensionner en 224*224 en conservant les proportions.
    Parameters
    ----------
    image : image à sauvegarder, obligatoire.
    Returns
    -------
    Répertoire de localisation de l'image redimensionnée.
    '''
    size = 224, 224
    # On charge l'image d'origine
    im = Image.open(image)
    file_dir = os.path.split(image)
    # L'un des côté de l'image fait 224, on garde le ratio original
    # (pas de déformation)
    im.thumbnail(size, Image.ANTIALIAS)
    # On enregistre dans un nouveau dossier l'image redimensionnée.
    im.save('data/Images_redim/' + file_dir[1])

    # Centrage
    im = Image.open('data/Images_redim/' + file_dir[1])
    width, height = im.size

    if height > width:
        img = Image.new('RGB', (224, 224), (255, 255, 255))  # white
        position_larg = int((height - width) / 2)
        img.paste(im, box=(position_larg, 0))
        img.save('data/Images_redim/' + file_dir[1])

    elif width > height:
        img = Image.new('RGB', (224, 224), (255, 255, 255))  # white
        position_haut = int((width - height) / 2)
        img.paste(im, box=(0, position_haut))
        img.save("data/Images_redim/" + file_dir[1])

    return 'data/Images_redim/' + file_dir[1]

def tsne_proj(input, data_tf):
    tfidf_tsne = TSNE(n_components=2).fit_transform(input)
    tfidf_tsne_df = pd.DataFrame(
        tfidf_tsne,
        columns=['tsne 2D - One', 'tsne 2D - Two'])
    tfidf_proj = pd.merge(data_tf,
                          tfidf_tsne_df,
                          left_index=True,
                          right_index=True,
                          how='inner')
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x='tsne 2D - One',
                    y='tsne 2D - Two',
                    hue='cat_lvl_1',
                    palette='Set2',
                    data=tfidf_proj)
    

def topic_table(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        t = (topic_idx)
        topics[t] = [feature_names[i] for i in top_words(topic, n_top_words)]
    return pd.DataFrame(topics)

def topics_tsne_proj(input, data_tf):
    model_df = pd.DataFrame(input)
    model_df['Best_Topic'] = model_df.idxmax(axis=1)
    model_tsne = TSNE(n_components=2).fit_transform(input)
    model_tsne_df = pd.DataFrame(
        model_tsne,
        columns=['tsne 2D - One', 'tsne 2D - Two'])
    model_proj = pd.merge(data_tf,
                          model_tsne_df,
                          left_index=True,
                          right_index=True,
                          how='inner')
    model_proj = pd.merge(model_proj,
                          model_df,
                          left_index=True,
                          right_index=True,
                          how='inner')
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x='tsne 2D - One',
                    y='tsne 2D - Two',
                    hue='cat_lvl_1',
                    style='Best_Topic',
                    palette='Set2',
                    s=70,
                    data=model_proj)

# extract features @path
def extract_features(img_path, model):

    # load image and convert it to grayscale
    # img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # load image
    img = cv2.imread(img_path)
    # detect key points and descriptors
    keypoints, descriptors = model.detectAndCompute(img, None)

    # returns keypoints and descriptors
    return keypoints, descriptors

def combine_and_cluster(text_features, img_features):
    # temp features dataframe
    features_to_clust = pd.DataFrame()
    # store results
    results = pd.DataFrame()
    # target to evaluate clustering with ARI
    target = data['cat_lvl_1']
    # define clusterer
    kmeans = KMeans(n_clusters=7)

    # loop over combinations
    # text as dictionary dim [0] of product, image as dim[1]
    for features in itertools.product(text_features, img_features):
        if (features[0] != 'None') or (features[1] != 'None'):
            if features[0] == 'None':
                name_iter = features[1]
                features_to_clust = img_features[features[1]]
            elif features[1] == 'None':
                name_iter = features[0]
                features_to_clust = text_features[features[0]]
            else:
                name_iter = features[0] + '+' + features[1]
                features_to_clust = np.concatenate(
                    (text_features[features[0]],
                     img_features[features[1]]),
                    axis=1)
            # fit and get labels
            kmeans.fit(features_to_clust)
            # store result
            results.loc[name_iter, 'ARI'] = adjusted_rand_score(
                target,
                kmeans.labels_)
    return results

def clusters_tsne_proj(input, labels, data_tf):
    data_tf['clusters'] = labels
    model_tsne = TSNE(n_components=2).fit_transform(input)
    model_tsne_df = pd.DataFrame(
        model_tsne,
        columns=['tsne 2D - One', 'tsne 2D - Two'])
    model_proj = pd.merge(data_tf,
                          model_tsne_df,
                          left_index=True,
                          right_index=True,
                          how='inner')
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x='tsne 2D - One',
                    y='tsne 2D - Two',
                    hue='cat_lvl_1',
                    style='clusters',
                    palette='Set2',
                    s=70,
                    data=model_proj)